<a href="https://colab.research.google.com/github/Harsh-Singh24/substitution/blob/main/Substitution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Substitution System

## Initializing

In [24]:
#Imporitng libraries
import gspread
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from google.colab import files


In [25]:
from google.colab import auth
auth.authenticate_user()

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import gspread

#Authenticate google sheet
from google.auth import default
creds, _ = default()

gs = gspread.authorize(creds)

In [28]:
#loading data
worksheet = gs.open('Preprocessing_final').sheet1
data = worksheet.get_all_values()
dataset = pd.DataFrame(data[1:], columns=data[0])


#Preprocessing Data
df1 = dataset[['Period','Class/section', 'Day', 'Time', 'Subject']]

df2 = dataset[['Period','Class/section', 'Day', 'Time', 'Teachers_Name']]
#replace Null values with N/A in column
df2['Teachers_Name'] = df2['Teachers_Name'].fillna('Not available')
# Function to split values in a4 column and convert to separate rows
def split_and_stack(df):
    stacked_df = (df.set_index(df.columns.drop('Subject',1).tolist())
                  .Subject.str.split('/', expand=True)
                  .stack()
                  .reset_index()
                  .rename(columns={0:'Subject'})
                  .loc[:, df.columns]
                 )
    return stacked_df

def split_and_stack2(df):
    stacked_df = (df.set_index(df.columns.drop('Teachers_Name',1).tolist())
                  .Teachers_Name.str.split('/', expand=True)
                  .stack()
                  .reset_index()
                  .rename(columns={0:'Teachers_Name'})
                  .loc[:, df.columns]
                 )
    return stacked_df
# Apply function to split

result1 = split_and_stack(df1)
result2 = split_and_stack2(df2)

#Merging the columns
dataset_timetable = pd.concat([result1, result2['Teachers_Name']], axis=1)
print(dataset_timetable.shape)

#setting display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

#Preprocessing for desired result
suffixes = ['Ms ','Mr ',' Mr','Ms.',' Ms','Mr.']
for suffix in suffixes:
    dataset_timetable['Teachers_Name'] = dataset_timetable['Teachers_Name'].str.replace(suffix, '')


dataset_timetable['Teachers_Name'] = dataset_timetable['Teachers_Name'].str.strip()
print(dataset_timetable.shape)


#fetching details
dataset_timetable['Teachers_Name'] = dataset_timetable['Teachers_Name'].apply(lambda x: str(x).upper())
dataset_timetable['Day'] = dataset_timetable['Day'].apply(lambda x: str(x).upper())
list_of_periods = list(dataset_timetable['Period'].unique())  #all the periods
list_of_days = list(dataset_timetable['Day'].unique()) #all the days
new = dataset_timetable[dataset_timetable['Day'] == datetime.today().strftime("%A").upper()] #todays time table

(1925, 6)
(1925, 6)


<ipython-input-28-a1e9a50d9ed4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Teachers_Name'] = df2['Teachers_Name'].fillna('Not available')


In [29]:
#Step 1: Finding free periods from Timetable for all the teachers

dict1 = {} #empty dict created to store teachers as key and free periods in values
for x in list(new['Teachers_Name'].unique()):  #for selecting a perticular subject
    new[new['Teachers_Name'] == x]            #todays timetable for selected teacher
    period_list = sorted(new[new['Teachers_Name'] == x]['Period'].to_list()) #periods of selected teacher in which classes are assigned
    set1 = set(list_of_periods)  #all the periods in school
    set2 = set(period_list)  #only periods for selected teachers in which classes are assigned

    difference = set1 - set2    #free periods for the selected teacher
    difference_list = list(difference) #free periods for the selected teacher converted into list
    dict1[x] = difference_list #assigning selected teacher and their free periods


#step 2: for a perticular period assiging all the free teachers available for that period

dict2 = {}  #created a empty to dict consist periods as key and available free teacher as key's values
for period in list_of_periods:  #selecting periods one by one
    list1 = [] #empty list to temporary the available free teacher for a selected period
    for teachers in dict1: #extracting teachers from dict1
        for xx in dict1[teachers]: #extracting the free periods for a perticular teacher
            if period == xx:
                list1.append(teachers) #if above looped period = to free period of selected teacher then this teacher assigned to the list
    dict2[period] = list1 #assigned the period as dict and teachers as its values


#step 3: Created a dataframe to store periods as header and available teachers in row

index = list(range(1,50))
dataset_freeperiod = pd.DataFrame() #create a empty dataframe to store
dataset_freeperiod['index'] = index  #assigned a column as index to insert 50 rows
for periods in dict2:
    lane = pd.Series(dict2[periods],name = periods) #for selected period store all the teachers in a series
    dataset_freeperiod[periods] = lane     #created a column with currently looped period and assigned the free teachers in it
dataset_freeperiod.drop(["index"],axis = 1, inplace = True) #deleted index column so my system does not confuse it as period and its values as teacher


#step 4: Finding unique classes of teachers

#dataset_timetable = pd.read_csv('Time table.csv') why would I need this ?
teachers_list = list(dataset_timetable['Teachers_Name'].unique())  #extracting all the teachers
index1 = list(range(1,50))
dataset_teacher_classes = pd.DataFrame() #created a empty dataframe to store Teachers_Name and thier classes
dataset_teacher_classes['index'] = index1
for teachers in teachers_list:
    unique_class = dataset_timetable[dataset_timetable['Teachers_Name'] == teachers]['Class/section'].unique() #classes for the teachers
    lane = pd.Series(unique_class,name = teachers) #store teacher and free period in a series
    dataset_teacher_classes[teachers] = lane #added teachers and free period in dataframe
dataset_teacher_classes.drop(["index"],axis = 1, inplace = True)  #deleted index column so my system does not confuse it as period and its values as teacher

#step 5A: loading booked dataset

try:
  worksheet = gs.open('Test Absent Teacher').worksheet('Booking_Slot')
  data1 = worksheet.get_all_values()
  Booked_slot_dataframe = pd.DataFrame(data1[1:], columns=data1[0])
except:
  pass

#Removing Booked teachers from given slot

for p in list(Booked_slot_dataframe['Booked_Period'].unique()):
  teachers_to_remove_from_availability = list(Booked_slot_dataframe[Booked_slot_dataframe['Booked_Period'] == p]['Teachers_Name'].unique())
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass


#step 6: Data preprocessing
#converting the names of teacher in dataset_teacher_classes into upper case
dataset_teacher_classes.columns = dataset_teacher_classes.columns.str.upper()
#converting all the data in upper string in our Teachers and day column
dataset_timetable['Teachers_Name'] = dataset_timetable['Teachers_Name'].apply(lambda x: str(x).upper())
dataset_timetable['Day'] = dataset_timetable['Day'].apply(lambda x: str(x).upper())
#converting all the data in upper string in our dataset
for col in dataset_freeperiod.columns.to_list():
    dataset_freeperiod[col] = dataset_freeperiod[col].apply(lambda x: str(x).upper())

## Checking


In [30]:
#step 7: Removing absent teachers and low preference from the availability
'''
for t in absent_teachers:
    #for column in dataset_freeperiod.columns.to_list():
    if (dataset_freeperiod == t).any().any():
        dataset_freeperiod.replace(t,'NAN', inplace=True)
'''
try:
  worksheet = gs.open('Test Absent Teacher').worksheet('Teachers_not_to_assign')
  data1 = worksheet.get_all_values()
  Teachers_not_to_assign_dataframe = pd.DataFrame(data1[1:], columns=data1[0])
  Teachers_not_to_assign = list(Teachers_not_to_assign_dataframe['Teachers_Name'])
except:
  Teachers_not_to_assign = ['ADITI MATHUR','NITA TIWARI','SOMVEER YADAV','KSHIPRA SETHI','POOJA ARORA','JATIN HASIJA','ANUPAM DIXIT','VIZUL CHAUDHARY','DEEPIKA SHARMA','PRATIMA JHA',
                          'GARIMA ARYA','SARU JAIN','VIJAY KUMAR','SANJAY GILL','RAKESH SHEORAN','ARUN GAUTAM','SUDHIR KUMAR','SONIA YADAV']


for t in Teachers_not_to_assign:
    if (dataset_freeperiod == t).any().any():
        dataset_freeperiod.replace(t,'NAN', inplace=True)

#Creating a dataframe to store the substitute teachers
#substitute = pd.DataFrame(columns=['Absent teacher','Period','Subject of absent', 'class','Substitute teacher'])
#extracting timetable for all the teachers only for today
#today_dataset_timetable = dataset_timetable[dataset_timetable['Day'] == datetime.today().strftime("%A").upper()]
today_dataset_timetable = new

##Substitition Processing

###Data loading

In [31]:
#load the absent teachers dataframe
worksheet = gs.open('Test Absent Teacher').worksheet('Todays_Absent_Teacher')
data = worksheet.get_all_values()
absent_teachers_dataframe = pd.DataFrame(data[1:], columns=data[0])

###Removing teachers from availabality

In [32]:
absent_teachers= list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'FDL']['Absent_Teacher'].unique())
#Removing teachers from availibility
for t in absent_teachers:
    #for column in dataset_freeperiod.columns.to_list():
    if (dataset_freeperiod == t).any().any():
        dataset_freeperiod.replace(t,'NAN', inplace=True)

#Removing absent_teachers_MSL from 1st and 2nd period
absent_teachers_MSL = list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'MSL']['Absent_Teacher'].unique())
period_MSL_A = ['Period 1','Period 2']
for p in period_MSL_A:
  teachers_to_remove_from_availability = absent_teachers_MSL
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass

#Removing absent_teachers_MSL from 9th Period
absent_teachers_ASL = list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'ASL']['Absent_Teacher'].unique())
period_ASL_A = ['Period 9']

for p in period_ASL_A:
  teachers_to_remove_from_availability = absent_teachers_ASL
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass

#Removing absent_teachers_MSL from FH slot
absent_teachers_FHL = list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'FHL']['Absent_Teacher'].unique())
#Removing absent_teachers_MSL from FH slot
period_FHL_A = ['Period 1','Period 2','Period 3','Period 4','Period 5']
for p in period_FHL_A:
  teachers_to_remove_from_availability = absent_teachers_FHL
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass

#Removing absent_teachers_MSL from FH slot
absent_teachers_SHL = list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'SHL']['Absent_Teacher'].unique())
period_SHL_A = ['Period 6','Period 7','Period 8','Period 9']
for p in period_SHL_A:
  teachers_to_remove_from_availability = absent_teachers_SHL
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass


### Substutuion for FDL

In [33]:
absent_teachers= list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'FDL']['Absent_Teacher'].unique())

#Creating a dataframe to store the substitute teachers
substitute_msl = pd.DataFrame(columns=['Absent teacher','Period','Subject of absent', 'class','Substitute teacher'])


#substitution

for teacher in absent_teachers:
    #store all the periods in which perticular absentee teaches
    periods = today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == teacher]['Period'].to_list()
    #store all the classes in which perticular absentee teaches
    classes = today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == teacher]['Class/section'].to_list()
    #print(teacher)
    subjects = today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == teacher]['Subject'].to_list()



    #extracting the periods one by one
    i = 0

    for period in periods:
        available_teachers = dataset_freeperiod[period].to_list()
        #available_teachers = dataset_freeperiod[dataset_freeperiod['Day']==datetime.today().strftime("%A").upper()][period].to_list()
        available_teachers = [x for x in available_teachers if x != 'NAN']

        c = classes[i]
        s = subjects[i]
        for available_teacher in available_teachers:
            #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
            if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
                #checking if available teacher share same class with absantee teacher class for that period
                if (c in list(dataset_teacher_classes[available_teacher])) and (s in new[new['Teachers_Name'] == available_teacher]['Subject'].to_list()): #Return the list of classes
                    substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                    break

        else:
            for available_teacher in available_teachers:
            #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
                if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
                #checking if available teacher share same class with absantee teacher class for that period
                    if (c in list(dataset_teacher_classes[available_teacher])): #Return the list of classes
                        substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                        break
            else:
              for available_teacher in available_teachers:
                if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7):
                  substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                  break
              else:
                substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,'Not available']
        i += 1

### Substutuion for MSL

In [34]:
absent_teachers_MSL = list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'MSL']['Absent_Teacher'].unique())

#Removing absent_teachers_MSL from 1st and 2nd period
'''
period_MSL_A = ['Period 1','Period 2']
for p in period_MSL_A:
  teachers_to_remove_from_availability = absent_teachers_MSL
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass
'''

#Substitution in MSL
for teacher in absent_teachers_MSL:
  #

  period_MSL = ['Period 1','Period 2']
  periods = []
  classes = []
  subjects = []

  periods_total = today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == teacher]['Period'].to_list()

  periods_to_assing = list(set(period_MSL).intersection(set(periods_total)))

  for p in periods_to_assing:
    periods.append(p)
    c = (today_dataset_timetable[(today_dataset_timetable['Teachers_Name'] == teacher) & (today_dataset_timetable['Period'] == p)]['Class/section']).values[0]
    classes.append(c)
    s = (today_dataset_timetable[(today_dataset_timetable['Teachers_Name'] == teacher) & (today_dataset_timetable['Period'] == p)]['Subject']).values[0]
    subjects.append(s)

  i = 0

  for period in periods:
      available_teachers = dataset_freeperiod[period].to_list()
      #available_teachers = dataset_freeperiod[dataset_freeperiod['Day']==datetime.today().strftime("%A").upper()][period].to_list()
      available_teachers = [x for x in available_teachers if x != 'NAN']
      c = classes[i]
      s = subjects[i]
      for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
        if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
            #checking if available teacher share same class with absantee teacher class for that period
            if (c in list(dataset_teacher_classes[available_teacher])) and (s in new[new['Teachers_Name'] == available_teacher]['Subject'].to_list()): #Return the list of classes
                substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                break
      else:
        for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
             if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
             #checking if available teacher share same class with absantee teacher class for that period
                if (c in list(dataset_teacher_classes[available_teacher])): #Return the list of classes
                    substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                    break
        else:
          for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
             if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7):
              substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
              break
          else:
            substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,'Not available']
      i += 1

### Substutuion for ASL

In [35]:
absent_teachers_ASL = list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'ASL']['Absent_Teacher'].unique())

#Removing absent_teachers_MSL from 9th Period
'''
period_ASL_A = ['Period 9']

for p in period_ASL_A:
  teachers_to_remove_from_availability = absent_teachers_ASL
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass
'''

#Substitution in ASL
for teacher in absent_teachers_ASL:
  #

  period_MSL = ['Period 9']
  periods = []
  classes = []
  subjects = []

  periods_total = today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == teacher]['Period'].to_list()

  periods_to_assing = list(set(period_MSL).intersection(set(periods_total)))

  for p in periods_to_assing:
    periods.append(p)
    c = (today_dataset_timetable[(today_dataset_timetable['Teachers_Name'] == teacher) & (today_dataset_timetable['Period'] == p)]['Class/section']).values[0]
    classes.append(c)
    s = (today_dataset_timetable[(today_dataset_timetable['Teachers_Name'] == teacher) & (today_dataset_timetable['Period'] == p)]['Subject']).values[0]
    subjects.append(s)

  i = 0

  for period in periods:
      available_teachers = dataset_freeperiod[period].to_list()
      #available_teachers = dataset_freeperiod[dataset_freeperiod['Day']==datetime.today().strftime("%A").upper()][period].to_list()
      available_teachers = [x for x in available_teachers if x != 'NAN']
      c = classes[i]
      s = subjects[i]
      for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
        if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
            #checking if available teacher share same class with absantee teacher class for that period
            if (c in list(dataset_teacher_classes[available_teacher])) and (s in new[new['Teachers_Name'] == available_teacher]['Subject'].to_list()): #Return the list of classes
                substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                break
      else:
        for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
             if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
             #checking if available teacher share same class with absantee teacher class for that period
                if (c in list(dataset_teacher_classes[available_teacher])): #Return the list of classes
                    substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                    break
        else:
          for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
             if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7):
              substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
              break
          else:
            substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,'Not available']
      i += 1

### Substutuion for FHL

In [36]:
absent_teachers_FHL = list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'FHL']['Absent_Teacher'].unique())

'''
#Removing absent_teachers_MSL from FH slot
period_FHL_A = ['Period 1','Period 2','Period 3','Period 4','Period 5']
for p in period_FHL_A:
  teachers_to_remove_from_availability = absent_teachers_FHL
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass
'''

#Substitution in FHL
for teacher in absent_teachers_FHL:
  #

  period_MSL = ['Period 1','Period 2','Period 3','Period 4','Period 5']
  periods = []
  classes = []
  subjects = []

  periods_total = today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == teacher]['Period'].to_list()

  periods_to_assing = list(set(period_MSL).intersection(set(periods_total)))

  for p in periods_to_assing:
    periods.append(p)
    c = (today_dataset_timetable[(today_dataset_timetable['Teachers_Name'] == teacher) & (today_dataset_timetable['Period'] == p)]['Class/section']).values[0]
    classes.append(c)
    s = (today_dataset_timetable[(today_dataset_timetable['Teachers_Name'] == teacher) & (today_dataset_timetable['Period'] == p)]['Subject']).values[0]
    subjects.append(s)

  i = 0

  for period in periods:
      available_teachers = dataset_freeperiod[period].to_list()
      #available_teachers = dataset_freeperiod[dataset_freeperiod['Day']==datetime.today().strftime("%A").upper()][period].to_list()
      available_teachers = [x for x in available_teachers if x != 'NAN']
      c = classes[i]
      s = subjects[i]
      for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
        if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
            #checking if available teacher share same class with absantee teacher class for that period
            if (c in list(dataset_teacher_classes[available_teacher])) and (s in new[new['Teachers_Name'] == available_teacher]['Subject'].to_list()): #Return the list of classes
                substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                break
      else:
        for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
             if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
             #checking if available teacher share same class with absantee teacher class for that period
                if (c in list(dataset_teacher_classes[available_teacher])): #Return the list of classes
                    substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                    break
        else:
          for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
             if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7):
              substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
              break
          else:
            substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,'Not available']
      i += 1

### Substutuion for SHL

In [37]:
absent_teachers_SHL = list(absent_teachers_dataframe[absent_teachers_dataframe['Leave _Type'] == 'SHL']['Absent_Teacher'].unique())

In [38]:
'''
#Removing absent_teachers_MSL from FH slot
period_SHL_A = ['Period 6','Period 7','Period 8','Period 9']
for p in period_SHL_A:
  teachers_to_remove_from_availability = absent_teachers_SHL
  for t in teachers_to_remove_from_availability:
    if (dataset_freeperiod[p] == t).any():
      dataset_freeperiod[p].replace(t,'NAN',inplace = True)
    else:
      pass
'''
#Substitution in SHL
for teacher in absent_teachers_SHL:
  #

  period_MSL = ['Period 6','Period 7','Period 8','Period 9']
  periods = []
  classes = []
  subjects = []

  periods_total = today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == teacher]['Period'].to_list()

  periods_to_assing = list(set(period_MSL).intersection(set(periods_total)))

  for p in periods_to_assing:
    periods.append(p)
    c = (today_dataset_timetable[(today_dataset_timetable['Teachers_Name'] == teacher) & (today_dataset_timetable['Period'] == p)]['Class/section']).values[0]
    classes.append(c)
    s = (today_dataset_timetable[(today_dataset_timetable['Teachers_Name'] == teacher) & (today_dataset_timetable['Period'] == p)]['Subject']).values[0]
    subjects.append(s)

  i = 0

  for period in periods:
      available_teachers = dataset_freeperiod[period].to_list()
      #available_teachers = dataset_freeperiod[dataset_freeperiod['Day']==datetime.today().strftime("%A").upper()][period].to_list()
      available_teachers = [x for x in available_teachers if x != 'NAN']
      c = classes[i]
      s = subjects[i]
      for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
        if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
            #checking if available teacher share same class with absantee teacher class for that period
            if (c in list(dataset_teacher_classes[available_teacher])) and (s in new[new['Teachers_Name'] == available_teacher]['Subject'].to_list()): #Return the list of classes
                substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                break
      else:
        for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
             if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7): #to_list() is very important here
             #checking if available teacher share same class with absantee teacher class for that period
                if (c in list(dataset_teacher_classes[available_teacher])): #Return the list of classes
                    substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
                    break
        else:
          for available_teacher in available_teachers:
        #Making a check that same teacher should not assinged to two classes at the same time and total periods for teachers should not be more that 8
             if (period not in substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) and ((len(substitute_msl[substitute_msl['Substitute teacher'] == available_teacher]['Period'].to_list()) + len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == available_teacher]['Period'].to_list())) < 7):
              substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,available_teacher]
              break
          else:
            substitute_msl.loc[len(substitute_msl.index)] = [teacher,period,s,c,'Not available']
      i += 1

##Final compiling

In [39]:
substitute = substitute_msl

# Display the final substitute DataFrame
substitute.sort_values(by=['Period','Absent teacher'],inplace = True)
#print(substitute)
#substitute.to_csv(f'class_6to12_{datetime.today().date()}/1csv')


#creating a dataframe to store the count of total periods of teacher
data = []

# Iterate over unique substitute teachers
for x in substitute['Substitute teacher'].unique():
    # Calculate total periods for the current substitute teacher
    total_periods = (len(substitute[substitute['Substitute teacher'] == x]['Period'].tolist()) +
                     len(today_dataset_timetable[today_dataset_timetable['Teachers_Name'] == x]['Period'].tolist()))

    # Append a dictionary containing teacher name and total periods to the list
    data.append({'Substitute teacher': x, 'Total_periods': total_periods})

# Create a DataFrame from the list of dictionaries
total_period_count = pd.DataFrame(data)

try:
  Final_result = pd.merge(substitute,total_period_count, on = 'Substitute teacher',how = 'left')

except:
  Final_result = pd.DataFrame(columns = ['Absent teacher', 'Period', 'Subject of absent', 'class', 'Substitute teacher','Total_periods'])

In [41]:
Final_result.rename(columns = {'Total_periods' : 'Total_work_load'},inplace = True)
Final_result['Date']  = datetime.today().date()